In [31]:
import cv2
from torch.utils.data import Dataset
import numpy as np
import torch
import random
from scipy import ndimage
import os
from archs_ucm_v2 import UCM_NetV2
from engineucm1 import train_one_epoch, val_one_epoch, test_one_epoch
from loader import isic_loader
from losses import GT_BceDiceLoss_new2
from utils_ucm import get_logger
from configs.config_setting import setting_config
from torch.optim import AdamW
from torch.utils.data import Dataset
import numpy as np
import torch
import random
from scipy import ndimage
from sklearn.model_selection import KFold ,train_test_split
from torch.utils.data import DataLoader



In [33]:

def dataset_normalized(imgs):
    imgs_std = np.std(imgs)
    imgs_mean = np.mean(imgs)
    imgs_normalized = (imgs - imgs_mean) / imgs_std
    for i in range(imgs.shape[0]):
        imgs_normalized[i] = (imgs_normalized[i] - np.min(imgs_normalized[i])) / (np.max(imgs_normalized[i]) - np.min(imgs_normalized[i]))
    return imgs_normalized

class ISICKFoldDataset(Dataset):
    def __init__(self, data, mask, train=True, augment=True):
        super().__init__()
        self.data = dataset_normalized(data.astype(np.float32))
        self.mask = (mask.astype(np.float32) / 255.0)
        self.train = train
        self.augment = augment

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        seg = self.mask[idx]

        if self.train and self.augment:
            if random.random() > 0.7:
                img, seg = self.random_rot_flip(img, seg)
            if random.random() > 0.7:
                img, seg = self.random_rotate(img, seg)

        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)
        seg = torch.tensor(seg, dtype=torch.float32)
        if seg.ndim == 2:
            seg = seg.unsqueeze(0)
        elif seg.ndim == 3:
            seg = seg.permute(2, 0, 1)

        return img, seg

    def random_rot_flip(self, img, label):
        k = np.random.randint(0, 4)
        img = np.rot90(img, k)
        label = np.rot90(label, k)
        axis = np.random.randint(0, 2)
        img = np.flip(img, axis=axis).copy()
        label = np.flip(label, axis=axis).copy()
        return img, label

    def random_rotate(self, img, label):
        angle = np.random.randint(20, 80)
        img = ndimage.rotate(img, angle, order=0, reshape=False)
        label = ndimage.rotate(label, angle, order=0, reshape=False)
        return img, label


In [34]:
# Initialize metric histories for this fold
train_metrics_history = {'loss': [], 'f1': [], 'acc': [], 'se': [], 'sp': []}
val_metrics_history = {'loss': [], 'f1': [], 'acc': [], 'se': [], 'sp': []}
test_metrics_history = {'loss': [], 'f1': [], 'acc': [], 'se': [], 'sp': []}


In [35]:
# ===== SAFE LOOKAHEAD WRAPPER =====
merged_data=np.load('merged_data2017.npy')
merged_mask=np.load('merged_mask2017.npy')
print(merged_data.shape)
k=5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# ===== SETUP =====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpu_ids = [0] if torch.cuda.is_available() else []

config = setting_config()
print(f'Using device: {device}')

from sklearn.model_selection import train_test_split
import numpy as np

# === Load original ISIC2017 test set (not part of merged data) ===
isic2017_test_dataset = isic_loader(path_Data='./data/ISIC2017/', train=False, Test=True)
isic2017_test_loader = DataLoader(isic2017_test_dataset, batch_size=1, shuffle=False, num_workers=config.num_workers)

# === Split merged dataset into train_val (1400) and test (600) ===
train_val_data, test_data, train_val_mask, test_mask = train_test_split(
    merged_data, merged_mask, test_size=600, random_state=42)

test_set = ISICKFoldDataset(test_data, test_mask, train=False)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=config.num_workers)

# === Manual 5-Fold with fixed 150-validation split ===
num_folds = 5
val_size = 150
all_indices = np.arange(train_val_data.shape[0])
np.random.seed(42)
np.random.shuffle(all_indices)

folds = []
for i in range(num_folds):
    val_start = i * val_size
    val_end = val_start + val_size
    val_idx = all_indices[val_start:val_end]
    train_idx = np.setdiff1d(all_indices, val_idx)
    folds.append((train_idx, val_idx))

# === Training Loop ===
fold = 1
for train_idx, val_idx in folds:
    print(f"\n===== 🔁 Fold {fold}/{num_folds} =====")

    train_set = ISICKFoldDataset(train_val_data[train_idx], train_val_mask[train_idx], train=True)
    val_set = ISICKFoldDataset(train_val_data[val_idx], train_val_mask[val_idx], train=False)

    train_loader = DataLoader(train_set, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
    val_loader = DataLoader(val_set, batch_size=1, shuffle=False, num_workers=config.num_workers)

    model = UCM_NetV2(1, 3, False)
    state_dict = torch.load('results/fine70/ultra__Friday_11_July_2025_12h_28m_08s/checkpoints_GT_BceDiceLoss_new2/best.pth', map_location='cpu')
    model.load_state_dict(state_dict)
    model = torch.nn.DataParallel(model.to(device), device_ids=gpu_ids)

    criterion = GT_BceDiceLoss_new2().to(device)
    optimizer = AdamW(model.parameters(), lr=2e-4, weight_decay=config.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)
    scaler = torch.cuda.amp.GradScaler() if config.amp else None

    log_dir = os.path.join(config.work_dir, f'log_fold_{fold}')
    checkpoint_dir = os.path.join(config.work_dir, f'checkpoints_fold_{fold}')
    os.makedirs(checkpoint_dir, exist_ok=True)
    logger = get_logger(f'fold_{fold}', log_dir)

    best_dice = 0.0
    min_loss = float('inf')
    start_epoch = 1
    num_fine_tune_epochs = 15

    for epoch in range(start_epoch, start_epoch + num_fine_tune_epochs):
        print(f"\n🔥 Fine-tuning Epoch {epoch}/{start_epoch + num_fine_tune_epochs - 1} (Fold {fold})")

        # TRAIN (assuming train_one_epoch returns loss, metrics dict)
        train_loss, train_metrics = train_one_epoch(train_loader, model, criterion, optimizer, scheduler,
                                                   epoch, logger, config, scaler=scaler, epoch_num=num_fine_tune_epochs)
        for key in train_metrics_history:
            train_metrics_history[key].append(train_metrics[key] if key != 'loss' else train_loss)

        # VALIDATION
        val_loss, val_metrics = val_one_epoch(val_loader, model, criterion, epoch,
                                              logger, config, epoch_num=num_fine_tune_epochs)
        for key in val_metrics_history:
            val_metrics_history[key].append(val_metrics[key] if key != 'loss' else val_loss)

        print(f"Validation - Loss: {val_loss:.4f}, Dice: {val_metrics['f1']:.4f}, Sensitivity: {val_metrics['se']:.4f}")

        # TEST (run every epoch)
        test_loss, test_metrics = test_one_epoch(test_loader, model, criterion, logger,
                                                 config, epoch=epoch, epoch_num=num_fine_tune_epochs)
        for key in test_metrics_history:
            test_metrics_history[key].append(test_metrics[key] if key != 'loss' else test_loss)

        print(f"Test - Loss: {test_loss:.4f}, Dice: {test_metrics['f1']:.4f}, Sensitivity: {test_metrics['se']:.4f}")

        # Save best model (optional, based on val_loss)
        if val_loss < min_loss:
            min_loss = val_loss
            print(f"✅ New minimum validation loss: {val_loss:.4f} — model saved.")
            torch.save(model.module.state_dict(), os.path.join(checkpoint_dir, 'fine_tuned_min_loss.pth'))


        fold += 1


(2000, 256, 256, 3)
Using device: cuda

===== 🔁 Fold 1/5 =====

🔥 Fine-tuning Epoch 1/15 (Fold 1)
train: epoch 1, iter:0, loss: 5.2229, lr: 0.000200
train: epoch 1, iter:20, loss: 2.5528, lr: 0.000200
train: epoch 1, iter:40, loss: 2.5093, lr: 0.000200
train: epoch 1, iter:60, loss: 2.4621, lr: 0.000200
train: epoch 1, iter:80, loss: 2.4167, lr: 0.000200
train: epoch 1, iter:100, loss: 2.3420, lr: 0.000200
train: epoch 1, iter:120, loss: 2.4154, lr: 0.000200
train: epoch 1, iter:140, loss: 2.4463, lr: 0.000200
[Train] Epoch 1 | Loss: 2.4692 | Dice: 0.0000 | Acc: 0.8154 | SE: 0.0000 | SP: 1.0000


100%|██████████| 150/150 [00:02<00:00, 57.31it/s]


val epoch: 1, loss: 2.1050, miou: 0.8726, f1_or_dsc: 0.9320, accuracy: 0.9735, specificity: 0.9869, sensitivity: 0.9190, confusion_matrix: [[7783580, 103360], [157338, 1786122]]
0.9734804280597967 0.8726326692132808 0.9190423265717231 0.9868947906284329
Validation - Loss: 2.1050, Dice: 0.9320, Sensitivity: 0.9190


100%|██████████| 600/600 [00:09<00:00, 62.04it/s]


test of best model, loss: 1.2640,miou: 0.8567163232583271, f1_or_dsc: 0.9228295270813227, accuracy: 0.970236078898112,                 specificity: 0.9872368931253673, sensitivity: 0.9011501257112952, confusion_matrix: [[31153433   402755]
 [  767610  6997802]]
Test - Loss: 1.2640, Dice: 0.9228, Sensitivity: 0.9012
✅ New minimum validation loss: 2.1050 — model saved.

🔥 Fine-tuning Epoch 2/15 (Fold 2)
train: epoch 2, iter:0, loss: 1.8607, lr: 0.000199
train: epoch 2, iter:20, loss: 2.3067, lr: 0.000199
train: epoch 2, iter:40, loss: 2.4209, lr: 0.000199


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(train_metrics_history['f1']) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_metrics_history['loss'], label='Train Loss', color='blue')
plt.plot(epochs, test_metrics_history['loss'], label='Test Loss', color='green')

plt.xlabel('Epoch')
plt.ylabel('acc')
plt.title('Loss per Epoch')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import torch
from torch.utils.data import DataLoader
import os
from archs_ucm_v2 import UCM_NetV2
from engineucm1 import train_one_epoch, val_one_epoch, test_one_epoch
from loader import isic_loader
from losses import GT_BceDiceLoss_new2
from utils_ucm import get_logger
from configs.config_setting import setting_config

# -------- Setup -------- #
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpu_ids = [0] if torch.cuda.is_available() else []

config = setting_config()
print(f'Using device: {device}')

# -------- Data -------- #
data_path = './data/ISIC2017/'

train_dataset = isic_loader(path_Data=data_path, train=True)
val_dataset = isic_loader(path_Data=data_path, train=False)
test_dataset = isic_loader(path_Data=data_path, train=False, Test=True)

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=config.num_workers)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=config.num_workers)

# -------- Model -------- #
model = UCM_NetV2(1, 3, False)

# -------- Load checkpoint -------- #
state_dict = torch.load('results/fine70/ultra__Friday_11_July_2025_12h_28m_08s/checkpoints_GT_BceDiceLoss_new2/best.pth')
new_state_dict = {k[len('module.'):] if k.startswith('module.') else k: v for k, v in state_dict.items()}
model.load_state_dict(new_state_dict)



# -------- Wrap with DataParallel and move to device -------- #
model = torch.nn.DataParallel(model.to(device), device_ids=gpu_ids, output_device=gpu_ids[0] if gpu_ids else None)

# -------- Training Setup -------- #
criterion = GT_BceDiceLoss_new2().to(device)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),   lr=2e-4, weight_decay=config.weight_decay)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)

scaler = torch.cuda.amp.GradScaler() if config.amp else None

# -------- Logger -------- #
log_dir = os.path.join(config.work_dir, 'log')
checkpoint_dir = os.path.join(config.work_dir, 'fine_tune_checkpoints')
os.makedirs(checkpoint_dir, exist_ok=True)
logger = get_logger('fine_tune', log_dir)

# -------- Fine-tuning Loop -------- #
best_dice = 0.0
best_se = 0.0
min_loss=float('inf')
start_epoch = 1
num_fine_tune_epochs = 30

# --- Initial test before training ---
test_loss, test_metrics = test_one_epoch(
    test_loader, model, criterion, logger, config,
    epoch=start_epoch, epoch_num=num_fine_tune_epochs
)
print(f"Initial Test - Loss: {test_loss:.4f}, Dice: {test_metrics['f1']:.4f}, Sensitivity: {test_metrics['se']:.4f}")

for epoch in range(start_epoch, start_epoch + num_fine_tune_epochs):
    print(f"\n🔥 Fine-tuning Epoch {epoch}/{start_epoch + num_fine_tune_epochs - 1}")

    # Train
    train_one_epoch(train_loader, model, criterion, optimizer, scheduler, epoch,
                    logger, config, scaler=scaler, epoch_num=num_fine_tune_epochs)

    # Validate
    val_loss, val_metrics = val_one_epoch(val_loader, model, criterion, epoch,
                                          logger, config, epoch_num=num_fine_tune_epochs)

    # Test


    # Extract validation metrics
    dice_score = val_metrics['f1']
    sensitivity = val_metrics['se']

    # Print metrics
    print(f"Validation - Loss: {val_loss:.4f}, Dice: {dice_score:.4f}, Sensitivity: {sensitivity:.4f}")
    print(f"Test       - Loss: {test_loss:.4f}, Dice: {test_metrics['f1']:.4f}, Sensitivity: {test_metrics['se']:.4f}")
    if val_loss < min_loss:
            min_loss = val_loss
            print(f"✅ New minimum validation loss: {val_loss:.4f} — model saved.")
            torch.save(model.module.state_dict(), os.path.join(checkpoint_dir, 'fine_tuned_min_loss.pth'))

            # Run test after improvement
            test_loss, test_metrics = test_one_epoch(test_loader, model, criterion, logger,
                                                     config, epoch=epoch, epoch_num=num_fine_tune_epochs)
            print(f"Test (best) - Loss: {test_loss:.4f}, Dice: {test_metrics['f1']:.4f}, Sensitivity: {test_metrics['se']:.4f}")


In [ ]:
pr

g

In [ ]:
# ===== SAFE LOOKAHEAD WRAPPER =====
merged_data=np.load('merged_data2018.npy')
merged_mask=np.load('merged_mask2018.npy')
print(merged_data.shape)
k=5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# ===== SETUP =====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpu_ids = [0] if torch.cuda.is_available() else []

config = setting_config()
print(f'Using device: {device}')

from sklearn.model_selection import train_test_split
import numpy as np

# === Load original ISIC2017 test set (not part of merged data) ===
isic2017_test_dataset = isic_loader(path_Data='./data/ISIC2018/', train=False, Test=True)
isic2017_test_loader = DataLoader(isic2017_test_dataset, batch_size=1, shuffle=False, num_workers=config.num_workers)

# === Split merged dataset into train_val (1400) and test (600) ===
train_val_data, test_data, train_val_mask, test_mask = train_test_split(
    merged_data, merged_mask, test_size=600, random_state=42)

test_set = ISICKFoldDataset(test_data, test_mask, train=False)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=config.num_workers)

# === Manual 5-Fold with fixed 150-validation split ===
num_folds = 5
val_size = 150
all_indices = np.arange(train_val_data.shape[0])
np.random.seed(42)
np.random.shuffle(all_indices)

folds = []
for i in range(num_folds):
    val_start = i * val_size
    val_end = val_start + val_size
    val_idx = all_indices[val_start:val_end]
    train_idx = np.setdiff1d(all_indices, val_idx)
    folds.append((train_idx, val_idx))

# === Training Loop ===
fold = 1
for train_idx, val_idx in folds:
    print(f"\n===== 🔁 Fold {fold}/{num_folds} =====")

    train_set = ISICKFoldDataset(train_val_data[train_idx], train_val_mask[train_idx], train=True)
    val_set = ISICKFoldDataset(train_val_data[val_idx], train_val_mask[val_idx], train=False)

    train_loader = DataLoader(train_set, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
    val_loader = DataLoader(val_set, batch_size=1, shuffle=False, num_workers=config.num_workers)

    model = UCM_NetV2(1, 3, False)
    state_dict = torch.load('results/fine2018/checkpoints_GT_BceDiceLoss_new2/best.pth', map_location='cpu')
    model.load_state_dict(state_dict)
    model = torch.nn.DataParallel(model.to(device), device_ids=gpu_ids)

    criterion = GT_BceDiceLoss_new2().to(device)
    optimizer = AdamW(model.parameters(), lr=2e-4, weight_decay=config.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)
    scaler = torch.cuda.amp.GradScaler() if config.amp else None

    log_dir = os.path.join(config.work_dir, f'log_fold_{fold}')
    checkpoint_dir = os.path.join(config.work_dir, f'checkpoints_fold_{fold}')
    os.makedirs(checkpoint_dir, exist_ok=True)
    logger = get_logger(f'fold_{fold}', log_dir)

    best_dice = 0.0
    min_loss = float('inf')
    start_epoch = 1
    num_fine_tune_epochs = 15

    for epoch in range(start_epoch, start_epoch + num_fine_tune_epochs):
        print(f"\n🔥 Fine-tuning Epoch {epoch}/{start_epoch + num_fine_tune_epochs - 1} (Fold {fold})")

        train_one_epoch(train_loader, model, criterion, optimizer, scheduler, epoch,
                        logger, config, scaler=scaler, epoch_num=num_fine_tune_epochs)

        val_loss, val_metrics = val_one_epoch(val_loader, model, criterion, epoch,
                                              logger, config, epoch_num=num_fine_tune_epochs)

        dice_score = val_metrics['f1']
        sensitivity = val_metrics['se']

        print(f"Validation - Loss: {val_loss:.4f}, Dice: {dice_score:.4f}, Sensitivity: {sensitivity:.4f}")

        if val_loss < min_loss:
            min_loss = val_loss
            print(f"✅ New minimum validation loss: {val_loss:.4f} — model saved.")
            torch.save(model.module.state_dict(), os.path.join(checkpoint_dir, 'fine_tuned_min_loss.pth'))

            # Run test after improvement
            test_loss, test_metrics = test_one_epoch(test_loader, model, criterion, logger,
                                                     config, epoch=epoch, epoch_num=num_fine_tune_epochs)
            print(f"Test (best) - Loss: {test_loss:.4f}, Dice: {test_metrics['f1']:.4f}, Sensitivity: {test_metrics['se']:.4f}")

    fold += 1

In [38]:
# ===== SAFE LOOKAHEAD WRAPPER =====
merged_data=np.load('ph2_images.npy')
merged_mask=np.load('ph2_masks.npy')
print(merged_data.shape)
k=5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# ===== SETUP =====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpu_ids = [0] if torch.cuda.is_available() else []

config = setting_config()
print(f'Using device: {device}')

from sklearn.model_selection import train_test_split
import numpy as np

# === Load original ISIC2017 test set (not part of merged data) ===


# === Split merged dataset into train_val (1400) and test (600) ===
# === Split merged dataset: 80% train_val, 20% test ===
train_val_data, test_data, train_val_mask, test_mask = train_test_split(
    merged_data, merged_mask, test_size=0.2, random_state=42)


test_set = ISICKFoldDataset(test_data, test_mask, train=False)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=config.num_workers)

# === Manual 5-Fold with fixed 150-validation split ===
num_folds = 5
val_size = 150
all_indices = np.arange(train_val_data.shape[0])
np.random.seed(42)
np.random.shuffle(all_indices)

folds = []
for i in range(num_folds):
    val_start = i * val_size
    val_end = val_start + val_size
    val_idx = all_indices[val_start:val_end]
    train_idx = np.setdiff1d(all_indices, val_idx)
    folds.append((train_idx, val_idx))

# === Training Loop ===
for fold_idx, (train_idx, val_idx) in enumerate(folds, 1):
    print(f"\n===== 🔁 Fold {fold_idx}/{num_folds} =====")

    # Reset metrics history for this fold
    train_metrics_history = {'loss': [], 'f1': [], 'acc': [], 'se': [], 'sp': []}
    val_metrics_history = {'loss': [], 'f1': [], 'acc': [], 'se': [], 'sp': []}
    test_metrics_history = {'loss': [], 'f1': [], 'acc': [], 'se': [], 'sp': []}

    train_set = ISICKFoldDataset(train_val_data[train_idx], train_val_mask[train_idx], train=True)
    val_set = ISICKFoldDataset(train_val_data[val_idx], train_val_mask[val_idx], train=False)

    train_loader = DataLoader(train_set, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
    val_loader = DataLoader(val_set, batch_size=1, shuffle=False, num_workers=config.num_workers)

    model = UCM_NetV2(1, 3, False)
    state_dict = torch.load('results/fine70/ultra__Friday_11_July_2025_12h_28m_08s/checkpoints_GT_BceDiceLoss_new2/best.pth',
                          map_location='cpu')
    model.load_state_dict(state_dict)
    model = torch.nn.DataParallel(model.to(device), device_ids=gpu_ids)

    criterion = GT_BceDiceLoss_new2().to(device)
    optimizer = AdamW(model.parameters(), lr=2e-4, weight_decay=config.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)
    scaler = torch.cuda.amp.GradScaler() if config.amp else None

    log_dir = os.path.join(config.work_dir, f'log_fold_{fold_idx}')
    checkpoint_dir = os.path.join(config.work_dir, f'checkpoints_fold_{fold_idx}')
    os.makedirs(checkpoint_dir, exist_ok=True)
    logger = get_logger(f'fold_{fold_idx}', log_dir)

    best_dice = 0.0
    min_loss = float('inf')
    start_epoch = 1
    num_fine_tune_epochs = 15

    for epoch in range(start_epoch, start_epoch + num_fine_tune_epochs):
        print(f"\n🔥 Fine-tuning Epoch {epoch}/{start_epoch + num_fine_tune_epochs - 1} (Fold {fold_idx})")

        # Train
        train_loss, train_metrics = train_one_epoch(train_loader, model, criterion, optimizer, scheduler,
                                                   epoch, logger, config, scaler=scaler,
                                                   epoch_num=num_fine_tune_epochs)
        for key in train_metrics_history:
            train_metrics_history[key].append(train_metrics[key] if key != 'loss' else train_loss)

        # Validation
        val_loss, val_metrics = val_one_epoch(val_loader, model, criterion, epoch,
                                              logger, config, epoch_num=num_fine_tune_epochs)
        for key in val_metrics_history:
            val_metrics_history[key].append(val_metrics[key] if key != 'loss' else val_loss)

        print(f"Validation - Loss: {val_loss:.4f}, Dice: {val_metrics['f1']:.4f}, "
              f"Sensitivity: {val_metrics['se']:.4f}")

        # Test
        test_loss, test_metrics = test_one_epoch(test_loader, model, criterion, logger,
                                                 config, epoch=epoch, epoch_num=num_fine_tune_epochs)
        for key in test_metrics_history:
            test_metrics_history[key].append(test_metrics[key] if key != 'loss' else test_loss)

        print(f"Test - Loss: {test_loss:.4f}, Dice: {test_metrics['f1']:.4f}, "
              f"Sensitivity: {test_metrics['se']:.4f}")

        # Save best model
        if val_loss < min_loss:
            min_loss = val_loss
            print(f"✅ New minimum validation loss: {val_loss:.4f} — model saved.")
            torch.save(model.module.state_dict(),
                      os.path.join(checkpoint_dir, 'fine_tuned_min_loss.pth'))


(200, 256, 256, 3)
Using device: cuda

===== 🔁 Fold 1/5 =====

🔥 Fine-tuning Epoch 1/15 (Fold 1)
train: epoch 1, iter:0, loss: 3.0950, lr: 0.000200
[Train] Epoch 1 | Loss: 2.5266 | Dice: 0.0000 | Acc: 0.5866 | SE: 0.0000 | SP: 1.0000


100%|██████████| 150/150 [00:02<00:00, 53.12it/s]


val epoch: 1, loss: 2.9521, miou: 0.8315, f1_or_dsc: 0.9080, accuracy: 0.9422, specificity: 0.9620, sensitivity: 0.8996, confusion_matrix: [[6456353, 255126], [313282, 2805639]]
0.9421785481769874 0.8315352453594063 0.8995543651150832 0.9619866202366182
Validation - Loss: 2.9521, Dice: 0.9080, Sensitivity: 0.8996


100%|██████████| 40/40 [00:00<00:00, 62.30it/s]


test of best model, loss: 1.4235,miou: 0.8442031309086003, f1_or_dsc: 0.9155207653212031, accuracy: 0.9462497711181641,                 specificity: 0.9584552075844114, sensitivity: 0.9199049129131445, confusion_matrix: [[1717038   74426]
 [  66477  763499]]
Test - Loss: 1.4235, Dice: 0.9155, Sensitivity: 0.9199
✅ New minimum validation loss: 2.9521 — model saved.

🔥 Fine-tuning Epoch 2/15 (Fold 1)
train: epoch 2, iter:0, loss: 2.2018, lr: 0.000199
[Train] Epoch 2 | Loss: 2.3146 | Dice: 0.0000 | Acc: 0.5976 | SE: 0.0000 | SP: 1.0000


...

100%|██████████| 10/10 [00:00<00:00, 57.00it/s]


val epoch: 5, loss: 1.6310, miou: 0.8996, f1_or_dsc: 0.9472, accuracy: 0.9555, specificity: 0.9679, sensitivity: 0.9388, confusion_matrix: [[364903, 12114], [17037, 261306]]
0.9555191040024482 0.899637467849287 0.9387913473630061 0.9678688228887082
Validation - Loss: 1.6310, Dice: 0.9472, Sensitivity: 0.9388


100%|██████████| 40/40 [00:00<00:00, 64.16it/s]


test of best model, loss: 0.9112,miou: 0.9133690495617645, f1_or_dsc: 0.9547233449511073, accuracy: 0.9714462280273437,                 specificity: 0.9809892914398504, sensitivity: 0.9508479763270263, confusion_matrix: [[1757407   34057]
 [  40795  789181]]
Test - Loss: 0.9112, Dice: 0.9547, Sensitivity: 0.9508
✅ New minimum validation loss: 1.6310 — model saved.

🔥 Fine-tuning Epoch 6/15 (Fold 2)
train: epoch 6, iter:0, loss: 1.4016, lr: 0.000187


KeyboardInterrupt: 